In [394]:
import os
import polars as pl

In [395]:
from src.najdi_tituly import najdi_tituly

In [396]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [397]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).collect()

In [398]:
df.filter(pl.col('datum').dt.year() <= 1990)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32


In [399]:
df.group_by('obdobi').agg(pl.col('text').unique().len()).sort(by='obdobi')

obdobi,text
i32,u32
1993,27889
1996,28485
1998,72830
2002,75512
2006,54101
2010,44970
2013,79941
2017,85811
2021,78859


In [400]:
df.filter(pl.col("id").is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Přítomni""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Předseda PSP M. Uhde, místopředsedové PSP J. Vlach, J. Kasal, K. Ledvinka, P. Tollner a 173 poslanců.""",1,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,16
"""Za vládu České republiky""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""předseda vlády V. Klaus, místopředseda vlády a ministr financí I. Kočárník, místopředseda vlády a ministr zemědělství J. Lux, místopředseda vlády J. K…",2,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,120
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, vážené dámy a pánové, zahajuji slavnostní schůzi Poslanecké sněmovny Parlamentu České republiky a všechn…",3,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,175
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené dámy a pánové, přistoupíme k prvému bodu pořadu naší schůze, a prosím, aby se slova ujal předseda Poslanecké sněmovny pan Milan Uhde.""",4,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,22
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážený pane předsedo vlády, vážení členové vlády, vážení členové diplomatického sboru, vážené paní poslankyně, vážení páni poslanci, vážení představit…",5,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1075
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi Poslanecké sněmovny panu Milanovi Uhdemu a nyní prosím, aby se ujal slova předseda vlády České republiky pan Václav Klaus.""",6,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,20
"""Předseda vlády ČR Václav Klaus""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, dámy a pánové, jistě nemusím zdůrazňovat, že jsem si vědom toho, že před vás předstupuji ve chvíli, kter…",7,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1852
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi vlády panu Václavu Klausovi. Nyní projednáme třetí bod našeho programu, kterým je Prosím předsedu Poslanecké sněmovny pana Milana …",8,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,27
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Prohlášení České národní rady k parlamentům a národům světa Česká národní rada, jako svobodně zvolený zákonodárný sbor České republiky, pamětliva tisí…",9,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,510


In [401]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Místopředseda PSP Karel Ledvinka""","""1993ps_stenprot_019schuz_s019029.htm""","""/sqw/hlasy.sqw?G=7268""","""Jedná se o 162. hlasování. Kdo tento návrh podporuje? Kdo je proti? Kdo se zdržel hlasování? Návrh nebyl přijat. Pro hlasovalo 20, proti 118, zdrželo …",2,"""ps""",1993,19,29,1994-06-02,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""19. schůze""",true,true,true,null,26
"""Poslankyně Eva Fischerová""","""1996ps_stenprot_017schuz_s017051.htm""","""/sqw/hlasy.sqw?G=10693""","""Pane předsedající, členové vlády, dámy a pánové, hlavním cílem navrhovaného usnesení je kritika strukturalizace lůžkového fondu a návrh na pozastavení…",8,"""ps""",1996,17,51,1997-12-10,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""17. schůze""",true,true,true,null,887
"""Místopředseda vlády ČR Pavel Bělobrádek""","""2013ps_stenprot_033schuz_s033168.htm""","""http://www.vlada.cz/cz/clenove-vlady/pri-uradu-vlady/pavel-belobradek/zivotopis/pavel-belobradek-115389/""","""Vy jste řekl, že něco podobného dělal Mussolini, což je pro mě srovnání s Mussolinim, nejsem sémantik, ale já to tak chápu a myslím si, že to je nefér…",3,"""ps""",2013,33,168,2015-10-09,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""33. schůze""",true,true,true,null,395
"""Místopředsedkyně PSP Petra Buzková""","""1996ps_stenprot_013schuz_s013034.htm""","""/sqw/detail.sqw?id=48&o=2""","""Zahajuji hlasování pořadové č. 171 a ptám se, kdo je pro tento návrh. Kdo je proti tomuto návrhu? Konstatuji, že v hlasování pořadové č. 171 se z přít…",18,"""ps""",1996,13,34,1997-09-05,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""13. schůze""",true,true,true,"""48&o=2""",37
"""Místopředseda vlády ČR, ministr vnitra a ministr zahraničních věcí Jan Hamáček""","""2017ps_stenprot_016schuz_s016086.htm""","""https://www.vlada.cz/cz/clenove-vlady/jan-hamacek-167024/""","""Děkuji. Vážený pane místopředsedo, vážené paní poslankyně, vážení páni poslanci, zákon o zpracování osobních údajů ruší stávající zákon o ochraně osob…",2,"""ps""",2017,16,86,2018-06-28,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""16. schůze""",true,true,true,null,132
"""Místopředsedkyně PSP Klára Dostálová""","""2021ps_stenprot_070schuz_s070116.htm""","""/sqw/detail.sqw?id=6430""","""Děkuji. To znamená přerušit písemnou interpelaci do doby přítomnosti člena vlády. Já zagonguji pro kolegy z předsálí, abychom požadavek paní poslankyn…",4,"""ps""",2021,70,116,2023-06-29,"""PČR, PS 2021-...""","""70. schůze""",true,true,true,"""6430""",201
"""Místopředseda PSP František Brožík""","""1998ps_stenprot_028schuz_s028149.htm""","""/sqw/p.sqw?P=432""","""Děkuji, pane poslanče. Otevírám obecnou rozpravu a táži se, kdo se do ní hlásí, protože písemné přihlášky nemám. Nehlásí se nikdo. Nejsou námitky k zr…",15,"""ps""",1998,28,149,2000-10-20,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""28. schůze""",true,true,true,null,45
"""Poslanec Miloslav Výborný""","""1993ps_stenprot_025schuz_s025055.htm""","""/sqw/hlasy.sqw?G=4554""","""Pane předsedající, prosil bych zpravodaje, aby vyložil, o jakém návrhu budeme hlasovat. Zda o vypuštění odkazu, aby tam tedy nebyl vůbec žádný odkaz, …",35,"""ps""",1993,25,55,1994-12-14,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""25. schůze""",true,true,true,null,58
"""Místopředseda PSP Jiří Honajzer""","""1996ps_stenprot_017schuz_s017058.htm""","""/sqw/hlasy.sqw?G=10693""","""Pan poslanec Ambrozek má faktickou poznámku.""",40,"""ps""",1996,17,58,1997-12-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""17. schůze""",true,true,true,null,5


In [402]:
df_mluvci = pl.read_parquet(os.path.join("data","mluvci.parquet"))

In [403]:
df_mluvci.sample(10)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""5914""",2010,"""5914_6.html""","""Mgr. Petr Gazdík""","""Zlínský""","""m""",2010-05-29,2013-08-28,"""TOP09""",null,"""1974""","[{""člen"",""Volební výbor"",""7. 7. 2010"",""28. 8. 2013""}, {""člen"",""Organizační výbor"",""7. 7. 2010"",""6. 12. 2011""}, … {""člen"",""Dozorčí rada Pozemkového fondu České republiky"",""6. 5. 2011"",""2. 12. 2011""}]",1974,null,"""Petr Gazdík""","[""Mgr.""]"
"""6150""",2021,"""6150_9.html""","""Ing. Andrej Babiš""","""Ústecký""","""m""",2021-10-09,null,"""ANO2011""",null,"""2. 9. 1954""","[{""člen"",""Poslanecký klub ANO 2011"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,1954-09-02,"""Andrej Babiš""","[""Ing.""]"
"""5457""",2006,"""5457_5.html""","""Robert Dušek""","""Liberecký""","""m""",2006-06-03,2009-07-13,"""ČSSD""",null,"""1967""","[{""člen"",""Výbor pro evropské záležitosti"",""12. 9. 2006"",""30. 10. 2008""}, {""člen"",""Kontrolní výbor"",""12. 9. 2006"",""13. 7. 2009""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""5. 6. 2006"",""13. 7. 2009""}]",1967,null,"""Robert Dušek""",[]
"""5769""",2006,"""5769_5.html""","""Zdenka Dopitová""","""Olomoucký""","""f""",2008-10-25,2010-06-03,"""ČSSD""",null,"""1948""","[{""člen"",""Mandátový a imunitní výbor"",""18. 2. 2009"",""3. 6. 2010""}, {""člen"",""Výbor pro sociální politiku"",""5. 11. 2009"",""3. 6. 2010""}, {""člen"",""Poslanecký klub České strany sociálně demokratické"",""25. 10. 2008"",""3. 6. 2010""}]",1948,null,"""Zdenka Dopitová""",[]
"""5937""",2010,"""5937_6.html""","""Lenka Kohoutová""","""Praha""","""f""",2010-05-29,2013-08-28,"""ODS""",null,"""1967""","[{""člen"",""Petiční výbor"",""7. 7. 2010"",""28. 8. 2013""}, {""člen"",""Kontrolní výbor"",""7. 7. 2010"",""18. 12. 2012""}, … {""člen"",""Skupina ČR - Vietnam"",""16. 12. 2010"",""28. 8. 2013""}]",1967,null,"""Lenka Kohoutová""",[]
"""5989""",2010,"""5989_6.html""","""Ing. Václav Zemek""","""Středočeský""","""m""",2010-05-29,2013-08-28,"""ČSSD""",null,"""1974""","[{""člen"",""Výbor pro životní prostředí"",""7. 7. 2010"",""28. 8. 2013""}, {""člen"",""Ústavně právní výbor"",""7. 7. 2010"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Irsko"",""9. 5. 2012"",""28. 8. 2013""}]",1974,null,"""Václav Zemek""","[""Ing.""]"
"""122""",1998,"""122_3.html""","""Ing. Hana Orgoníková""","""Východočeský""","""f""",1998-06-20,2002-06-20,"""ČSSD""",null,"""1946""","[{""člen"",""Organizační výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Petiční výbor"",""22. 7. 1998"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Indie"",""19. 5. 1999"",""20. 6. 2002""}]",1946,null,"""Hana Orgoníková""","[""Ing.""]"
"""6809""",2021,"""6809_9.html""","""Ing. Jarmila Levko""","""Liberecký""","""f""",2022-02-15,2022-03-23,"""PirSTAN""","""STAN""","""14. 5. 1969""","[{""člen"",""Rozpočtový výbor"",""23. 3. 2022"",null}, {""člen"",""Zahraniční výbor"",""7. 9. 2022"",null}, … {""předsed"",""Skupina ČR - Republika Myanmarský svaz"",""11. 12. 2024"",null}]",1969,1969-05-14,"""Jarmila Levko""","[""Ing.""]"
"""193""",2002,"""193_4.html""","""JUDr. Miloslav Výborný""","""Pardubický""","""m""",2002-06-15,2003-06-02,"""K""","""KDU-ČSL""","""1952""","[{""člen"",""Ústavně právní výbor"",""16. 7. 2002"",""2. 6. 2003""}, {""předsed"",""Ústavně právní výbor"",""17. 7. 2002"",""2. 6. 2003""}, … {""člen"",""Skupina ČR - Slovensko"",""5. 3. 2003"",""2. 6. 2003""}]",1952,null,"""Miloslav Výborný""","[""JUDr.""]"


In [404]:
df_mluvci.sort(
    by=['id','jmeno']
).unique(
    subset=['id','jmeno'], keep='first'
)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6466""",2021,"""6466_9.html""","""Ing. Jana Krutáková""","""Jihomoravský""","""f""",2021-10-09,2021-11-23,"""PirSTAN""","""STAN""","""21. 6. 1962""","[{""člen"",""Zemědělský výbor"",""23. 11. 2021"",null}, {""člen"",""Výbor pro životní prostředí"",""23. 11. 2021"",null}, … {""předsed"",""Skupina ČR - Rakousko"",""2. 3. 2022"",null}]",1962,1962-06-21,"""Jana Krutáková""","[""Ing.""]"
"""379""",2002,"""379_4.html""","""Ing. Jaromír Kohlíček, CSc.""","""Ústecký""","""m""",2002-06-15,2004-07-20,"""KSČM""",null,"""1953""","[{""člen"",""Výbor pro evropskou integraci"",""16. 7. 2002"",""12. 5. 2004""}, {""člen"",""Zahraniční výbor"",""16. 7. 2002"",""20. 7. 2004""}, … {""člen"",""Skupina ČR - Japonsko"",""2. 9. 2025"",""20. 7. 2004""}]",1953,null,"""Jaromír Kohlíček""","[""Ing."", ""CSc.""]"
"""6268""",2013,"""6268_7.html""","""Ing. Vlastimil Gabrhel""","""Jihomoravský""","""m""",2014-04-30,2017-10-26,"""ČSSD""",null,"""1978""","[{""člen"",""Zemědělský výbor"",""14. 5. 2014"",""26. 10. 2017""}, {""člen"",""Hospodářský výbor"",""14. 5. 2014"",""26. 10. 2017""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""30. 4. 2014"",""26. 10. 2017""}]",1978,null,"""Vlastimil Gabrhel""","[""Ing.""]"
"""5980""",2010,"""5980_6.html""","""Ing. Roman Váňa""","""Olomoucký""","""m""",2010-05-29,2013-08-28,"""ČSSD""",null,"""1966""","[{""člen"",""Výbor pro obranu a bezpečnost"",""7. 7. 2010"",""6. 12. 2011""}, {""člen"",""Výbor pro bezpečnost"",""13. 12. 2011"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Arménie"",""14. 12. 2012"",""28. 8. 2013""}]",1966,null,"""Roman Váňa""","[""Ing.""]"
"""360""",1998,"""360_3.html""","""JUDr. Eva Dundáčková""","""Středočeský""","""f""",1998-06-20,2002-06-20,"""ODS""",null,"""1964""","[{""člen"",""Ústavně právní výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Podvýbor pro mládež"",""24. 9. 1998"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Nizozemsko"",""13. 6. 2001"",""20. 6. 2002""}]",1964,null,"""Eva Dundáčková""","[""JUDr.""]"
"""6786""",2021,"""6786_9.html""","""MUDr. Róbert Teleky, MBA""","""Zlínský""","""m""",2021-10-09,null,"""SPOLU""","""KDU-ČSL""","""30. 6. 1967""","[{""člen"",""Hospodářský výbor"",""23. 11. 2021"",null}, {""člen"",""Výbor pro zdravotnictví"",""23. 11. 2021"",null}, … {""člen"",""Dozorčí rada VZP"",""18. 2. 2022"",null}]",1967,1967-06-30,"""Róbert Teleky""","[""MUDr."", ""MBA""]"
"""6587""",2017,"""6587_8.html""","""Mgr. Bc. Petr Venhoda""","""Praha""","""m""",2018-10-01,2021-10-20,"""ANO2011""",null,"""1986""","[{""člen"",""Rozpočtový výbor"",""24. 10. 2018"",""20. 10. 2021""}, {""člen"",""Mandátový a imunitní výbor"",""7. 12. 2018"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Izrael"",""25. 9. 2019"",""20. 10. 2021""}]",1986,null,"""Petr Venhoda""","[""Mgr."", ""Bc.""]"
"""6235""",2013,"""6235_7.html""","""Ing. Roman Procházka""","""Karlovarský""","""m""",2013-10-26,2017-10-26,"""ANO2011""",null,"""1968""","[{""člen"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""21. 10. 2015""}, … {""předsed"",""Skupina ČR - Spolková republika Německo"",""10. 1. 2017"",""26. 10. 2017""}]",1968,null,"""Roman Procházka""","[""Ing.""]"
"""406""",2006,"""406_5.html""","""Ing. Alena Páralová""","""Pardubický""","""f""",2006-06-03,2010-06-03,"""ODS""",null,"""1948""","[{""člen"",""Organizační výbor"",""12. 9. 2006"",""5. 11. 2009""}, {""člen"",""Výbor pro sociální politiku"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Poslanecký klub Občanské demokratické strany"",""8. 6. 2006"",""3. 6. 2010""}]",1948,null,"""Alena Páralová""","[""Ing.""]"


In [405]:
df_mluvci.filter(pl.col("jmeno").str.contains("Andrej Babiš"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6150""",2013,"""6150_7.html""","""Ing. Andrej Babiš""","""Praha""","""m""",2013-10-26,2017-10-26,"""ANO2011""",null,"""1954""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""29. 1. 2014""}, {""člen"",""Kontrolní výbor"",""7. 6. 2017"",""26. 10. 2017""}, … {null,""Vláda České republiky"",""29. 1. 2014"",""24. 5. 2017""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2017,"""6150_8.html""","""Ing. Andrej Babiš""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""ANO2011""",null,"""1954""","[{""člen"",""Poslanecký klub ANO 2011"",""24. 10. 2017"",""20. 10. 2021""}, {""předsed"",""Vláda České republiky"",""6. 12. 2017"",""27. 6. 2018""}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2021,"""6150_9.html""","""Ing. Andrej Babiš""","""Ústecký""","""m""",2021-10-09,null,"""ANO2011""",null,"""2. 9. 1954""","[{""člen"",""Poslanecký klub ANO 2011"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,1954-09-02,"""Andrej Babiš""","[""Ing.""]"


In [406]:
df_mluvci.filter(pl.col("jmeno").str.contains("Tomio Okamura"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6105""",2013,"""6105_7.html""","""Tomio Okamura""","""Středočeský""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1972""","[{""člen"",""Organizační výbor"",""26. 3. 2014"",""30. 4. 2015""}, {""člen"",""Podvýbor pro Ústavu České republiky a parlamentní procedury"",""18. 3. 2014"",""26. 10. 2017""}, … {""člen"",""Skupina ČR - Japonsko"",""13. 2. 2014"",""26. 10. 2017""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2017,"""6105_8.html""","""Tomio Okamura""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1972""","[{""člen"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, … {""předsed"",""Skupina ČR - Afghánská islámská republika"",""9. 3. 2020"",""20. 10. 2021""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2021,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]


In [407]:
df.sort(by='datum').filter(pl.col('mluvci').str.contains('Petr Fiala'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",9,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Paní Hana Zejdová se sice v roce 1993 umístila na prvním místě na mistrovství Evropy v bezmotorovém létání, v následujících letech už nebyla tak úspěš…",1,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,false,null,145
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,n

In [408]:
df.filter(pl.col("id") == "6074")

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové, odpověděl bych na otázku paní poslankyně Jourové. Chápu paní poslankyni Jourovou. Doufám, že chápat mohu i bez prostřednictví paní před…",1,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",264
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji. Na to jsou jednoznačná čísla, kterými to lze doložit, a tyto prostředky jsou prostě potřeba. (Potlesk části poslanců.)""",3,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",18
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004073.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové. Rád bych prostřednictvím paní předsedající sdělil paní poslankyni Jourové, že pokud zmiňovala nadstandardní spolupráci České republiky …",14,"""ps""",2013,4,73,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",107
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004074.htm""","""/sqw/detail.sqw?id=6074""","""Pokud jde o evropské strukturální fondy a jejich čerpání, mám zkušenosti ze všech rolí, a to z role řešitele, z role příjemce i z role řídícího orgánu…",1,"""ps""",2013,4,74,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,false,"""6074""",101
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005002.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji, pane předsedo. Dámy a pánové, je o mně známo, že jsem vcelku důsledným obhájcem parlamentní demokracie, a tudíž bych měl mít radost, že se sch…",2,"""ps""",2013,5,2,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",484
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005004.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, pan předseda Filip mě tady pochválil jako politologa, já předpokládám, že to udělal proto, aby hned na začátku os…",5,"""ps""",2013,5,4,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",114
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005005.htm""","""/sqw/detail.sqw?id=6074""","""Pan předseda Faltýnek tady zmiňoval naše chyby a věci, kterých jsme se dopustili, když jsme byli ve vládě. My jsme za ty chyby zaplatili a zaplatili j…",1,"""ps""",2013,5,5,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,false,"""6074""",256
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005008.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, celá ta debata mě jenom utvrdila v tom, že můj příměr s řidičákem je naprosto přesný. Pan Babiš je možná dobrý ři…",6,"""ps""",2013,5,8,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",145
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005031.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedající, dámy a pánové, rád bych vás upozornil na některé problémy, které se pojí s tímto návrhem. Jak už tady bylo řečeno, poslanci …",1,"""ps""",2013,5,31,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",537


In [409]:
df_mluvci.filter(pl.col("jmeno").str.contains("Fiala"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""5458""",2006,"""5458_5.html""","""Ing. Jaroslav Fiala, CSc.""","""Karlovarský""","""m""",2006-06-03,2010-06-03,"""ČSSD""",null,"""1956""","[{""člen"",""Výbor pro obranu"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Kontrolní výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""5. 6. 2006"",""3. 6. 2010""}]",1956,null,"""Jaroslav Fiala""","[""Ing."", ""CSc.""]"
"""5459""",2006,"""5459_5.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2006-06-03,2010-06-03,"""ODS""",null,"""1969""","[{""člen"",""Výbor pro životní prostředí"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Rozpočtový výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Čína"",""21. 3. 2007"",""3. 6. 2010""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2010,"""5459_6.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2010-05-29,2013-08-28,"""ODS""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""7. 7. 2010"",""18. 12. 2012""}, {""člen"",""Výbor pro životní prostředí"",""7. 7. 2010"",""28. 8. 2013""}, … {""místopředsed"",""Skupina ČR - Ruská federace"",""3. 2. 2011"",""28. 8. 2013""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2013,"""5459_7.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""30. 4. 2015""}, {""člen"",""Organizační výbor"",""4. 12. 2013"",""26. 3. 2014""}, … {""předsed"",""Skupina ČR - Turecká republika"",""13. 2. 2014"",""26. 10. 2017""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2013,"""6074_7.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2013-10-26,2017-10-26,"""ODS""",null,"""1964""","[{""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""26. 10. 2017""}, {""místopředsed"",""Výbor pro evropské záležitosti"",""5. 12. 2013"",""26. 10. 2017""}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""6. 11. 2013"",""9. 12. 2014""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2017,"""5459_8.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1969""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Hospodářský výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Brazílie"",""18. 3. 2019"",""20. 10. 2021""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2017,"""6074_8.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1964""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""24. 10. 2017"",""20. 10. 2021""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2021,"""5459_9.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2021-10-09,null,"""SPD""",null,"""24. 7. 1969""","[{""člen"",""Hospodářský výbor"",""23. 11. 2021"",null}, {""člen"",""Organizační výbor"",""23. 11. 2021"",null}, … {""předsed"",""Poslanecký klub Svoboda a přímá demokracie"",""12. 10. 2021"",null}]",1969,1969-07-24,"""Radim Fiala""","[""Ing.""]"
"""6074""",2021,"""6074_9.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""1. 9. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""28. 11. 2021"",null}]",1964,1964-09-01,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"


In [410]:
df.filter(pl.col("mluvci").str.contains("Petr Fiala"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",9,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Paní Hana Zejdová se sice v roce 1993 umístila na prvním místě na mistrovství Evropy v bezmotorovém létání, v následujících letech už nebyla tak úspěš…",1,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,false,null,145
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,n

## Sloučení

In [412]:
df = df.join(df_mluvci, on=['id','obdobi'], how='left')

In [413]:
klic = df_mluvci.select(pl.col(["jmeno","id"])).unique()
klic.sample(3)

jmeno,id
str,str
"""Ing. Pavel Hrnčíř""","""371"""
"""Jaroslav Zavadil""","""6157"""
"""MUDr. Jozef Kochan""","""5375"""


In [414]:
df.group_by('obdobi').agg(pl.col('pocet_slov').sum()).sort(by='obdobi')

obdobi,pocet_slov
i32,u32
1993,3708695
1996,3760392
1998,8712710
2002,9457716
2006,7266422
2010,6591019
2013,11264523
2017,13108730
2021,15623462


In [415]:
df.filter(pl.col('obdobi') == 2021).group_by('id').agg(pl.col('pocet_slov').sum()).sort(by='pocet_slov',descending=True).join(klic, how='left',on='id')

id,pocet_slov,jmeno
str,u32,str
null,1987939,null
"""6105""",1128473,"""Tomio Okamura"""
"""6150""",1093438,"""Ing. Andrej Babiš"""
"""6621""",614754,"""doc. Ing. Karel Havlíček, Ph.D., MBA"""
"""6545""",490503,"""JUDr. Alena Schillerová, Ph.D."""
"""6473""",463584,"""PhDr. Olga Richterová, Ph.D."""
"""6178""",451329,"""Ing. Jan Bartošek"""
"""6463""",405574,"""Ing. Aleš Juchelka"""
"""6147""",386879,"""Ing. Markéta Pekarová Adamová"""


In [416]:
import altair as alt

In [417]:
df.sort(by='datum').tail(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Souhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čísle…",3,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",58,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Souhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Je to hlasování s pořadovým…",4,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",77,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Stanovisko? (Zpravodaj: Stanovisko výboru - nedoporučující.) Pan ministr? (Ministr: Nesouhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tl…",5,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",61,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",6,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",66,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",7,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",58,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""

In [418]:
df.filter(pl.col("id") == "6105").sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum())

datum,pocet_slov
date,u32
2013-12-10,258
2013-12-12,595
2014-02-18,618
2014-03-19,1938
2014-03-25,4767
2014-04-30,1466
2014-05-15,2377
2014-06-10,1477
2014-06-11,2252


In [419]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Andrej Babiš")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [420]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Tomio Okamura")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [421]:
alt.Chart(
    df.sort(by="datum").group_by_dynamic(index_column="datum",every="1mo").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [422]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Marek Benda")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [424]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null()).select(pl.col('mluvci')).unique()

mluvci
str
"""Místopředseda vlády a ministr zdravotnictví ČR Vlastimil Válek"""
"""Ministr pro evropské záležitosti ČR Martin Dvořák"""
"""Ministryně spravedlnosti ČR Eva Decroix"""
"""Předseda vlády ČR Andrej Babiš"""
"""Ministr kultury ČR Martin Baxa"""
"""Ministr pro evropské záležitosti ČR Mikuláš Bek"""
"""Prezident České republiky Petr Pavel"""
"""Ministryně pro vědu a výzkum ČR Helena Langšádlová"""
"""Předseda vlády ČR Petr Fiala"""


In [425]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001009.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Pane předsedo, děkuji. Já jenom technická. Hlasoval jsem pro, ale nepodařilo se mi přihlásit. Tak to jenom na upřesnění.""",3,"""ps""",2021,1,9,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,18,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda vlády, ministr průmyslu a obchodu a ministr dopravy ČR Karel Havlíček""","""2021ps_stenprot_001schuz_s001010.htm""","""https://www.vlada.cz/cz/clenove-vlady/karel-havlicek-173244/""","""Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Nebudu v tuto chvíli mluvit k těm právním náležitostem, určitě o tom ještě v p…",5,"""ps""",2021,1,10,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,266,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda vlády, ministr průmyslu a obchodu a ministr dopravy ČR Karel Havlíček""","""2021ps_stenprot_001schuz_s001011.htm""","""https://www.vlada.cz/cz/clenove-vlady/karel-havlicek-173244/""","""Můžete se nám vysmát a věcně argumentovat tím, že jste nám nabídli dva místopředsedy, případně pár výborů, a to zase my můžeme reagovat, že s místopře…",1,"""ps""",2021,1,11,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,634,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001011.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Především bych vám všem chtěla pogratulovat ke zvolení a záro…",3,"""ps""",2021,1,11,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,541,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001012.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Výkladu čl. 32 v tom smyslu, že se jedná o ustanovení o neslučitelnosti, nikoliv o nevolitelnosti, svědčí také ustanovení čl. 2 odst. 4 a čl. 2 odst. …",1,"""ps""",2021,1,12,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,526,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001012.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci, já se nebudu vyjadřovat k právním otázkám, já se vyjádřím k d…",3,"""ps""",2021,1,12,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,459,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""I pokud se jedná o otázku ŘSD, jsou zde také vidět silné a věcné kroky docenta Havlíčka, zejména při schvalování záměru na transformaci na novou státn…",1,"""ps""",2021,1,13,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,185,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministryně pro místní rozvoj ČR Klára Dostálová""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz

In [426]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id') == '6105')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Místopředseda PSP Tomio Okamura""","""2021ps_stenprot_001schuz_s001001.htm""","""/sqw/detail.sqw?id=6105""","""(Schůze zahájena ve 14.00 hodin.)""",1,"""ps""",2021,1,1,2021-11-08,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,"""6105""",4,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002001.htm""","""/sqw/detail.sqw?id=6105""","""Vážené dámy a pánové, navrhuji jako první bod dnešní schůze Sněmovny mimořádný bod programu s názvem Diskuse o nepřijatelných opatřeních souvisejících…",5,"""ps""",2021,2,1,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",129,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002002.htm""","""/sqw/detail.sqw?id=6105""","""Já osobně k tomu dodávám, že omezování svobody, omezování pohybu, šikanování podnikatelů a občanů navíc poškozuje občany i na jejich zdraví. Pokud se …",1,"""ps""",2021,2,2,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",1295,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002003.htm""","""/sqw/detail.sqw?id=6105""","""Dovolte dále citovat nález rozšířeného senátu Nejvyššího správního soudu. Cituji: ""Abyste mohli zasahovat do základních práv v důsledku mimořádných op…",1,"""ps""",2021,2,3,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",1137,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Navíc vláda Andreje Babiše s účinností od včerejšího dne zavádí takzvaný bavorský model, podle kterého zdraví neočkovaní lidé nesmí chodit na volnočas…",1,"""ps""",2021,2,4,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",372,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Ne, ne, já jsem říkal, že nejprve bych to navrhl - já jsem dal dvě varianty... (Pekarová: Ano, prosím.) ...abych dal prostor k tomu po

In [427]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Přítomni""","""1993ps_stenprot_029schuz_s029007.htm""","""/sqw/hlasy.sqw?G=4193""","""Předseda PSP M. Uhde, místopředseda PSP J. Vlach, místopředseda PSP J. Kasal, místopředseda PSP K. Ledvinka, místopředseda PSP P. Tollner a 176 poslan…",1,"""ps""",1993,29,7,1995-03-15,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""29. schůze""",true,true,true,null,22,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Vít Rakušan""","""2017ps_stenprot_013schuz_s013103.htm""","""/sqw/detail.sqw?id=6524""","""Děkuji za slovo. Možná spíše dodatek než samotná otázka na pana ministra. Pane ministře, já bych vás tedy chtěl poprosit o to, zda by nebylo v rámci k…",5,"""ps""",2017,13,103,2018-05-24,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""13. schůze""",true,true,true,"""6524""",133,"""6524_8.html""","""Mgr. Vít Rakušan""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""STAN""",null,"""1978""","[{""člen"",""Výbor pro bezpečnost"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Zahraniční výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Bahrajnské království"",""28. 11. 2019"",""20. 10. 2021""}]",1978,null,"""Vít Rakušan""","[""Mgr.""]"
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_036schuz_s036001.htm""","""/sqw/hlasy.sqw?G=1861""","""Zítra jako první bod odpoledne?""",25,"""ps""",1993,36,1,1995-10-31,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""36. schůze""",true,true,true,null,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Předseda PSP Jan Hamáček""","""2013ps_stenprot_023schuz_s023180.htm""","""/sqw/detail.sqw?id=5462""","""Zahajuji hlasování. Ptám se, kdo je pro, ať stiskne tlačítko a zvedne ruku. Kdo je proti tomuto návrhu? Hlasování má číslo 102, přihlášeno je 176, pro…",1,"""ps""",2013,23,180,2014-12-10,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""23. schůze""",true,true,false,"""5462""",81,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda PSP Petr Fiala""","""2017ps_stenprot_030schuz_s030051.htm""","""/sqw/detail.sqw?id=6074""","""Paní ministryně? (Nesouhlasné.) Zahajuji hlasování. Kdo je pro. Kdo je proti? Hlasování číslo 171, přihlášeno 185 poslanců, pro 80, proti 85. Návrh ne…",20,"""ps""",2017,30,51,2019-05-29,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""30. schůze""",true,true,true,"""6074""",23,"""6074_8.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1964""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""24. 10. 2017"",""20. 10. 2021""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""Poslanec Adam Rykala""","""2010ps_stenprot_020schuz_s020062.htm""","""/sqw/detail.sqw?id=5960""","""Děkuji za slovo, paní předsedkyně. Vážené kolegyně, vážení kolegové, spousta věcí tady už dneska padla. Mě mrzí, že ministr financí není v tuto chvíli…",3,"""ps""",2010,20,62,2011-07-13,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""20. schůze""",true,true,true,"""5960""",901,"""5960_6.html""","""Ing. Adam Rykala""","""Moravskoslezský""","""m""",2010-05-29,2013-08-28,"""ČSSD""",null,"""1986""","[{""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""7. 7. 2010"",""28. 8. 2013""}, {null,""Výbor pro věd

In [428]:
df.select(pl.col("pocet_slov")).sum()

pocet_slov
u32
79493669


"Naše pilotní studie ukazuje, že normální řeč plynula rychlostí 1,75 slov za sekundu (105 slov/min.). Bylo ji však možné zrychlit při rutinních úlohách (mě- síce) až na 3 slova za sekundu (180 slov/min.).
"

In [430]:
df.select(pl.col("pocet_slov")).sum().item() / (105 * 60 * 8 * 365.25)

4.318289766739459